In [1]:
import pandas as pd
from datetime import datetime,date, time

In [2]:
#pd.show_versions()
print(pd.__version__)

1.2.4


In [3]:
import pyspark
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
spark = SparkSession.builder.appName("bitcoin_pyspark_kraken").getOrCreate()
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/14 00:18:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
btc_sdf = spark.read.csv("data/bitcoin_price_all.csv", header=True, inferSchema=True)

In [7]:
btc_sdf.show(3)

+-------------------+------+------+------+------+------+------+
|               date|  open|  high|   low| close|volume|trades|
+-------------------+------+------+------+------+------+------+
|2013-10-06 00:00:00| 122.0| 122.0| 122.0| 122.0|   0.1|     1|
|2013-10-07 00:00:00|123.61|123.61|123.61|123.61|   0.1|     1|
|2013-10-08 00:00:00|123.91|124.19| 123.9|124.18|3.9916|     4|
+-------------------+------+------+------+------+------+------+
only showing top 3 rows



In [8]:
btc_sdf.orderBy(btc_sdf.date.desc()).show(3)

+-------------------+-------+-------+-------+-------+-------------+------+
|               date|   open|   high|    low|  close|       volume|trades|
+-------------------+-------+-------+-------+-------+-------------+------+
|2022-10-10 00:00:00|19443.5|19520.0|19108.3|19237.8|2348.44749395| 12890|
|2022-10-09 00:00:00|19420.0|19559.5|19335.0|19443.5| 903.92350217|  9334|
|2022-10-08 00:00:00|19532.8|19615.0|19249.7|19417.5|1337.85077668| 10018|
+-------------------+-------+-------+-------+-------+-------------+------+
only showing top 3 rows



In [9]:
btc_sdf.count()   # only bitcoin, not included other coins

3280

In [10]:
btc_sdf.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: double (nullable = true)
 |-- trades: integer (nullable = true)



In [11]:
btc_sdf = btc_sdf.withColumn("date", to_date("date"))
btc_sdf.show(5)

+----------+---------+---------+------+------+------+------+
|      date|     open|     high|   low| close|volume|trades|
+----------+---------+---------+------+------+------+------+
|2013-10-06|    122.0|    122.0| 122.0| 122.0|   0.1|     1|
|2013-10-07|   123.61|   123.61|123.61|123.61|   0.1|     1|
|2013-10-08|   123.91|   124.19| 123.9|124.18|3.9916|     4|
|2013-10-09|124.01687|124.01687|123.84|123.84| 2.823|     3|
|2013-10-10|   125.85|   125.86|125.85|125.86|   2.0|     2|
+----------+---------+---------+------+------+------+------+
only showing top 5 rows



In [12]:
btc_sdf.printSchema()

root
 |-- date: date (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: double (nullable = true)
 |-- trades: integer (nullable = true)



### Grabbing Data

**take()**

In [13]:
btc_sdf.take(2)

[Row(date=datetime.date(2013, 10, 6), open=122.0, high=122.0, low=122.0, close=122.0, volume=0.1, trades=1),
 Row(date=datetime.date(2013, 10, 7), open=123.61, high=123.61, low=123.61, close=123.61, volume=0.1, trades=1)]

**first()**

In [14]:
btc_sdf.first()  # only show first row

Row(date=datetime.date(2013, 10, 6), open=122.0, high=122.0, low=122.0, close=122.0, volume=0.1, trades=1)

**head()**

In [15]:
btc_sdf.head(2)   # use head with specify number

[Row(date=datetime.date(2013, 10, 6), open=122.0, high=122.0, low=122.0, close=122.0, volume=0.1, trades=1),
 Row(date=datetime.date(2013, 10, 7), open=123.61, high=123.61, low=123.61, close=123.61, volume=0.1, trades=1)]

**tail()**

In [16]:
btc_sdf.tail(2)

[Row(date=datetime.date(2022, 10, 9), open=19420.0, high=19559.5, low=19335.0, close=19443.5, volume=903.92350217, trades=9334),
 Row(date=datetime.date(2022, 10, 10), open=19443.5, high=19520.0, low=19108.3, close=19237.8, volume=2348.44749395, trades=12890)]

**select**

In [17]:
btc_sdf.select(["date", "close"]).show(5)

+----------+------+
|      date| close|
+----------+------+
|2013-10-06| 122.0|
|2013-10-07|123.61|
|2013-10-08|124.18|
|2013-10-09|123.84|
|2013-10-10|125.86|
+----------+------+
only showing top 5 rows



**filter()**

In [18]:
btc_sdf.filter(btc_sdf.date =="2022-09-30").show()

+----------+-------+-------+-------+-------+-------------+------+
|      date|   open|   high|    low|  close|       volume|trades|
+----------+-------+-------+-------+-------+-------------+------+
|2022-09-30|19600.0|20177.1|19150.0|19425.2|8327.92704339| 39279|
+----------+-------+-------+-------+-------+-------------+------+



In [19]:
btc_sdf.filter((btc_sdf.date >="2022-08-30") & (btc_sdf.close > 20000)).show()

+----------+-------+-------+-------+-------+-------------+------+
|      date|   open|   high|    low|  close|       volume|trades|
+----------+-------+-------+-------+-------+-------------+------+
|2022-08-31|19814.2|20473.4|19814.2|20052.6|4013.35676203| 24216|
|2022-09-01|20049.4|20195.0|19560.0|20127.8|2829.58668692| 23301|
|2022-09-09|19322.5|21585.1|19309.4|21375.4|6607.96000464| 35413|
|2022-09-10|21373.3|21800.0|21121.6|21650.4|2229.18631665| 18887|
|2022-09-11|21656.6|21850.2|21363.2|21844.6|1949.29969214| 15491|
|2022-09-12|21835.6|22475.0|21564.6|22401.3|4241.18154876| 26130|
|2022-09-13|22397.8|22790.0|19863.2|20173.6|9152.50302423| 44068|
|2022-09-14|20172.8|20525.0|19619.5|20233.0|4155.45394503| 25335|
|2022-09-17|19795.4|20197.8|19757.0|20118.0|1889.86114721| 15534|
|2022-10-04|19631.4|20475.0|19508.2|20347.9|5668.96124874| 24242|
|2022-10-05|20340.9|20360.9|19754.3|20162.7|3851.61226863| 19609|
+----------+-------+-------+-------+-------+-------------+------+



In [20]:
btc_sdf.filter(btc_sdf.date >="2022-08-30")\
       .filter(btc_sdf.open > 20000)       \
       .filter(btc_sdf.close > 20000)      \
       .filter(btc_sdf.high > 20000)       \
       .filter(btc_sdf.low > 20000)        \
       .show()

+----------+-------+-------+-------+-------+-------------+------+
|      date|   open|   high|    low|  close|       volume|trades|
+----------+-------+-------+-------+-------+-------------+------+
|2022-09-10|21373.3|21800.0|21121.6|21650.4|2229.18631665| 18887|
|2022-09-11|21656.6|21850.2|21363.2|21844.6|1949.29969214| 15491|
|2022-09-12|21835.6|22475.0|21564.6|22401.3|4241.18154876| 26130|
+----------+-------+-------+-------+-------+-------------+------+



**collect()** : takes up memory space

In [21]:
btc_sdf.filter(btc_sdf.date >="2022-10-1").collect()

[Row(date=datetime.date(2022, 10, 1), open=19425.1, high=19478.6, low=19172.0, close=19316.1, volume=1680.93419445, trades=19051),
 Row(date=datetime.date(2022, 10, 2), open=19316.1, high=19393.7, low=18937.0, close=19057.8, volume=1799.11964791, trades=12608),
 Row(date=datetime.date(2022, 10, 3), open=19060.4, high=19700.0, low=18985.7, close=19639.6, volume=3507.0515593, trades=20153),
 Row(date=datetime.date(2022, 10, 4), open=19631.4, high=20475.0, low=19508.2, close=20347.9, volume=5668.96124874, trades=24242),
 Row(date=datetime.date(2022, 10, 5), open=20340.9, high=20360.9, low=19754.3, close=20162.7, volume=3851.61226863, trades=19609),
 Row(date=datetime.date(2022, 10, 6), open=20163.1, high=20449.8, low=19872.7, close=19955.8, volume=3542.0619383, trades=20390),
 Row(date=datetime.date(2022, 10, 7), open=19955.8, high=20054.7, low=19338.4, close=19532.8, volume=3678.17182192, trades=19341),
 Row(date=datetime.date(2022, 10, 8), open=19532.8, high=19615.0, low=19249.7, close=

### Create New Columns

In [22]:
# from pyspark.sql.functions import (dayofmonth, month, year, weekofyear, format_number)

In [23]:
btc_sdf.withColumn("day"  , dayofmonth(btc_sdf["date"]))  \
       .withColumn('month', month(btc_sdf["date"]))       \
       .withColumn("year" , year(btc_sdf["date"]))        \
       .withColumn("week" , weekofyear(btc_sdf["date"]))  \
       .show(10)

+----------+---------+---------+---------+---------+-----------+------+---+-----+----+----+
|      date|     open|     high|      low|    close|     volume|trades|day|month|year|week|
+----------+---------+---------+---------+---------+-----------+------+---+-----+----+----+
|2013-10-06|    122.0|    122.0|    122.0|    122.0|        0.1|     1|  6|   10|2013|  40|
|2013-10-07|   123.61|   123.61|   123.61|   123.61|        0.1|     1|  7|   10|2013|  41|
|2013-10-08|   123.91|   124.19|    123.9|   124.18|     3.9916|     4|  8|   10|2013|  41|
|2013-10-09|124.01687|124.01687|   123.84|   123.84|      2.823|     3|  9|   10|2013|  41|
|2013-10-10|   125.85|   125.86|   125.85|   125.86|        2.0|     2| 10|   10|2013|  41|
|2013-10-12|    127.5|    127.5|    127.0|    127.0|        4.0|     3| 12|   10|2013|  41|
|2013-10-13| 131.8408| 131.8408| 131.8408| 131.8408|        0.1|     1| 13|   10|2013|  41|
|2013-10-14|    134.8|   135.59|133.36726|   135.59|12.00575337|    11| 14|   10

### Rename Column

In [24]:
btc_sdf.withColumnRenamed("trades", "num_trades").show(5)

+----------+---------+---------+------+------+------+----------+
|      date|     open|     high|   low| close|volume|num_trades|
+----------+---------+---------+------+------+------+----------+
|2013-10-06|    122.0|    122.0| 122.0| 122.0|   0.1|         1|
|2013-10-07|   123.61|   123.61|123.61|123.61|   0.1|         1|
|2013-10-08|   123.91|   124.19| 123.9|124.18|3.9916|         4|
|2013-10-09|124.01687|124.01687|123.84|123.84| 2.823|         3|
|2013-10-10|   125.85|   125.86|125.85|125.86|   2.0|         2|
+----------+---------+---------+------+------+------+----------+
only showing top 5 rows



### Order By 

In [25]:
btc_sdf.orderBy(btc_sdf.date.desc()).show(10)

+----------+-------+-------+-------+-------+-------------+------+
|      date|   open|   high|    low|  close|       volume|trades|
+----------+-------+-------+-------+-------+-------------+------+
|2022-10-10|19443.5|19520.0|19108.3|19237.8|2348.44749395| 12890|
|2022-10-09|19420.0|19559.5|19335.0|19443.5| 903.92350217|  9334|
|2022-10-08|19532.8|19615.0|19249.7|19417.5|1337.85077668| 10018|
|2022-10-07|19955.8|20054.7|19338.4|19532.8|3678.17182192| 19341|
|2022-10-06|20163.1|20449.8|19872.7|19955.8| 3542.0619383| 20390|
|2022-10-05|20340.9|20360.9|19754.3|20162.7|3851.61226863| 19609|
|2022-10-04|19631.4|20475.0|19508.2|20347.9|5668.96124874| 24242|
|2022-10-03|19060.4|19700.0|18985.7|19639.6| 3507.0515593| 20153|
|2022-10-02|19316.1|19393.7|18937.0|19057.8|1799.11964791| 12608|
|2022-10-01|19425.1|19478.6|19172.0|19316.1|1680.93419445| 19051|
+----------+-------+-------+-------+-------+-------------+------+
only showing top 10 rows



In [26]:
btc_sdf.orderBy(btc_sdf.close.desc()).select('date','close').show(5)

+----------+-------+
|      date|  close|
+----------+-------+
|2021-11-08|67559.8|
|2021-11-09|66950.0|
|2021-10-20|66035.8|
|2021-11-14|65495.0|
|2021-11-10|64969.0|
+----------+-------+
only showing top 5 rows



### Sort

In [27]:
btc_sdf.sort(btc_sdf.date.desc()).show(5)

+----------+-------+-------+-------+-------+-------------+------+
|      date|   open|   high|    low|  close|       volume|trades|
+----------+-------+-------+-------+-------+-------------+------+
|2022-10-10|19443.5|19520.0|19108.3|19237.8|2348.44749395| 12890|
|2022-10-09|19420.0|19559.5|19335.0|19443.5| 903.92350217|  9334|
|2022-10-08|19532.8|19615.0|19249.7|19417.5|1337.85077668| 10018|
|2022-10-07|19955.8|20054.7|19338.4|19532.8|3678.17182192| 19341|
|2022-10-06|20163.1|20449.8|19872.7|19955.8| 3542.0619383| 20390|
+----------+-------+-------+-------+-------+-------------+------+
only showing top 5 rows



In [28]:
btc_sdf.sort(btc_sdf.close.desc()).select('date','close').show(5)

+----------+-------+
|      date|  close|
+----------+-------+
|2021-11-08|67559.8|
|2021-11-09|66950.0|
|2021-10-20|66035.8|
|2021-11-14|65495.0|
|2021-11-10|64969.0|
+----------+-------+
only showing top 5 rows



### Aggregation | Group By

In [29]:
btc_sdf.groupBy(year(btc_sdf["date"])).mean("close").show()

+----------+------------------+
|year(date)|        avg(close)|
+----------+------------------+
|      2018| 7523.407671232872|
|      2015| 272.3238580821917|
|      2022|31172.531802120153|
|      2013| 528.8305436470587|
|      2014| 531.4999854929574|
|      2019| 7364.964109589046|
|      2020|11116.068579180335|
|      2016| 568.2766947267767|
|      2017| 3972.502397260274|
|      2021|47426.086301369854|
+----------+------------------+



In [30]:
mean_close_yearly = btc_sdf.groupBy(year(btc_sdf["date"])).mean("close")

In [31]:
mean_close_yearly = mean_close_yearly.select("year(date)", format_number("avg(close)", 2))
mean_close_yearly .show()

+----------+----------------------------+
|year(date)|format_number(avg(close), 2)|
+----------+----------------------------+
|      2018|                    7,523.41|
|      2015|                      272.32|
|      2022|                   31,172.53|
|      2013|                      528.83|
|      2014|                      531.50|
|      2019|                    7,364.96|
|      2020|                   11,116.07|
|      2016|                      568.28|
|      2017|                    3,972.50|
|      2021|                   47,426.09|
+----------+----------------------------+



In [32]:
mean_close_yearly.columns

['year(date)', 'format_number(avg(close), 2)']

In [33]:
mean_close_yearly = mean_close_yearly.withColumnRenamed('year(date)', "year")\
                                     .withColumnRenamed('format_number(avg(close), 2)', "average_close")

In [34]:
mean_close_yearly.orderBy(mean_close_yearly.year).show()

+----+-------------+
|year|average_close|
+----+-------------+
|2013|       528.83|
|2014|       531.50|
|2015|       272.32|
|2016|       568.28|
|2017|     3,972.50|
|2018|     7,523.41|
|2019|     7,364.96|
|2020|    11,116.07|
|2021|    47,426.09|
|2022|    31,172.53|
+----+-------------+



In [35]:
btc_sdf.show(5)

+----------+---------+---------+------+------+------+------+
|      date|     open|     high|   low| close|volume|trades|
+----------+---------+---------+------+------+------+------+
|2013-10-06|    122.0|    122.0| 122.0| 122.0|   0.1|     1|
|2013-10-07|   123.61|   123.61|123.61|123.61|   0.1|     1|
|2013-10-08|   123.91|   124.19| 123.9|124.18|3.9916|     4|
|2013-10-09|124.01687|124.01687|123.84|123.84| 2.823|     3|
|2013-10-10|   125.85|   125.86|125.85|125.86|   2.0|     2|
+----------+---------+---------+------+------+------+------+
only showing top 5 rows



In [36]:
btc_sdf.agg(F.mean(btc_sdf["close"])).show()

+------------------+
|        avg(close)|
+------------------+
|11471.362422106697|
+------------------+



In [37]:
btc_sdf.groupBy(year(btc_sdf["date"]))  \
       .agg(F.mean("close"))            \
       .show()

+----------+------------------+
|year(date)|        avg(close)|
+----------+------------------+
|      2018| 7523.407671232872|
|      2015| 272.3238580821917|
|      2022|31172.531802120153|
|      2013| 528.8305436470587|
|      2014| 531.4999854929574|
|      2019| 7364.964109589046|
|      2020|11116.068579180335|
|      2016| 568.2766947267767|
|      2017| 3972.502397260274|
|      2021|47426.086301369854|
+----------+------------------+



In [38]:
btc_yearly = btc_sdf.groupBy(year(btc_sdf["date"]))

In [39]:
btc_yearly.agg(F.mean("close")).show()

+----------+------------------+
|year(date)|        avg(close)|
+----------+------------------+
|      2018| 7523.407671232872|
|      2015| 272.3238580821917|
|      2022|31172.531802120153|
|      2013| 528.8305436470587|
|      2014| 531.4999854929574|
|      2019| 7364.964109589046|
|      2020|11116.068579180335|
|      2016| 568.2766947267767|
|      2017| 3972.502397260274|
|      2021|47426.086301369854|
+----------+------------------+



In [40]:
btc_sdf.groupBy(year(btc_sdf["date"]))   \
       .agg(F.mean(btc_sdf["close"]),    \
            F.min(btc_sdf["close"]),     \
            F.max(btc_sdf["close"]))     \
       .orderBy(year(btc_sdf["date"]))   \
       .show()

+----------+------------------+----------+----------+
|year(date)|        avg(close)|min(close)|max(close)|
+----------+------------------+----------+----------+
|      2013| 528.8305436470587|     122.0|  1165.425|
|      2014| 531.4999854929574| 311.00761| 919.11176|
|      2015| 272.3238580821917|     175.0|    464.98|
|      2016| 568.2766947267767|    364.49|   978.001|
|      2017| 3972.502397260274|   789.884|   19356.9|
|      2018| 7523.407671232872|    3180.1|   17088.0|
|      2019| 7364.964109589046|    3360.3|   12933.7|
|      2020|11116.068579180335|    4850.0|   28959.2|
|      2021|47426.086301369854|   29398.9|   67559.8|
|      2022|31172.531802120153|   18469.2|   47738.3|
+----------+------------------+----------+----------+



In [41]:
btc_yearly_stat = btc_sdf.groupBy(year(btc_sdf["date"]))\
       .agg(F.mean(btc_sdf["close"]),                   \
            F.min(btc_sdf["close"]),                    \
            F.max(btc_sdf["close"]))

In [42]:
btc_yearly_stat.show(5)

+----------+------------------+----------+----------+
|year(date)|        avg(close)|min(close)|max(close)|
+----------+------------------+----------+----------+
|      2018| 7523.407671232872|    3180.1|   17088.0|
|      2015| 272.3238580821917|     175.0|    464.98|
|      2022|31172.531802120153|   18469.2|   47738.3|
|      2013| 528.8305436470587|     122.0|  1165.425|
|      2014| 531.4999854929574| 311.00761| 919.11176|
+----------+------------------+----------+----------+
only showing top 5 rows



In [43]:
btc_yearly_stat = btc_yearly_stat.withColumnRenamed("year(date)","year")          \
                                 .withColumnRenamed("avg(close)","average_close") \
                                 .withColumnRenamed("min(close)","min_close")     \
                                 .withColumnRenamed("max(close)","max_close") 

In [44]:
btc_yearly_stat.show(5)

+----+------------------+---------+---------+
|year|     average_close|min_close|max_close|
+----+------------------+---------+---------+
|2018| 7523.407671232872|   3180.1|  17088.0|
|2015| 272.3238580821917|    175.0|   464.98|
|2022|31172.531802120153|  18469.2|  47738.3|
|2013| 528.8305436470587|    122.0| 1165.425|
|2014| 531.4999854929574|311.00761|919.11176|
+----+------------------+---------+---------+
only showing top 5 rows



In [45]:
btc_yearly_stat.sort(btc_yearly_stat.max_close.desc()).show()

+----+------------------+---------+---------+
|year|     average_close|min_close|max_close|
+----+------------------+---------+---------+
|2021|47426.086301369854|  29398.9|  67559.8|
|2022|31172.531802120153|  18469.2|  47738.3|
|2020|11116.068579180335|   4850.0|  28959.2|
|2017| 3972.502397260274|  789.884|  19356.9|
|2018| 7523.407671232872|   3180.1|  17088.0|
|2019| 7364.964109589046|   3360.3|  12933.7|
|2013| 528.8305436470587|    122.0| 1165.425|
|2016| 568.2766947267767|   364.49|  978.001|
|2014| 531.4999854929574|311.00761|919.11176|
|2015| 272.3238580821917|    175.0|   464.98|
+----+------------------+---------+---------+



## Save to | write to file

In [46]:
btc_sdf.show(5)

+----------+---------+---------+------+------+------+------+
|      date|     open|     high|   low| close|volume|trades|
+----------+---------+---------+------+------+------+------+
|2013-10-06|    122.0|    122.0| 122.0| 122.0|   0.1|     1|
|2013-10-07|   123.61|   123.61|123.61|123.61|   0.1|     1|
|2013-10-08|   123.91|   124.19| 123.9|124.18|3.9916|     4|
|2013-10-09|124.01687|124.01687|123.84|123.84| 2.823|     3|
|2013-10-10|   125.85|   125.86|125.85|125.86|   2.0|     2|
+----------+---------+---------+------+------+------+------+
only showing top 5 rows



**to_csv file**

In [47]:
btc_sdf.write.mode("overwrite").options(header=True).csv("data/btc_spark_df.csv")

## PySpark Using SQL

In [48]:
btc_sdf.createOrReplaceTempView("btc_sql")

In [49]:
spark.sql("SELECT * FROM btc_sql").show()

+----------+---------+---------+---------+---------+-----------+------+
|      date|     open|     high|      low|    close|     volume|trades|
+----------+---------+---------+---------+---------+-----------+------+
|2013-10-06|    122.0|    122.0|    122.0|    122.0|        0.1|     1|
|2013-10-07|   123.61|   123.61|   123.61|   123.61|        0.1|     1|
|2013-10-08|   123.91|   124.19|    123.9|   124.18|     3.9916|     4|
|2013-10-09|124.01687|124.01687|   123.84|   123.84|      2.823|     3|
|2013-10-10|   125.85|   125.86|   125.85|   125.86|        2.0|     2|
|2013-10-12|    127.5|    127.5|    127.0|    127.0|        4.0|     3|
|2013-10-13| 131.8408| 131.8408| 131.8408| 131.8408|        0.1|     1|
|2013-10-14|    134.8|   135.59|133.36726|   135.59|12.00575337|    11|
|2013-10-15|    135.8|    153.0|133.87975|133.87975|        4.3|     5|
|2013-10-16|    143.0|   146.14|137.63758|   138.09|     20.229|    17|
|2013-10-17|137.62448|   144.73|   137.52|143.79522|      5.777|

In [50]:
spark.sql("""
SELECT *, 
      dayofweek(date) AS dayofweek,
      month(date)     AS month,
      year(date)      AS year
FROM btc_sql
""").show(10)

+----------+---------+---------+---------+---------+-----------+------+---------+-----+----+
|      date|     open|     high|      low|    close|     volume|trades|dayofweek|month|year|
+----------+---------+---------+---------+---------+-----------+------+---------+-----+----+
|2013-10-06|    122.0|    122.0|    122.0|    122.0|        0.1|     1|        1|   10|2013|
|2013-10-07|   123.61|   123.61|   123.61|   123.61|        0.1|     1|        2|   10|2013|
|2013-10-08|   123.91|   124.19|    123.9|   124.18|     3.9916|     4|        3|   10|2013|
|2013-10-09|124.01687|124.01687|   123.84|   123.84|      2.823|     3|        4|   10|2013|
|2013-10-10|   125.85|   125.86|   125.85|   125.86|        2.0|     2|        5|   10|2013|
|2013-10-12|    127.5|    127.5|    127.0|    127.0|        4.0|     3|        7|   10|2013|
|2013-10-13| 131.8408| 131.8408| 131.8408| 131.8408|        0.1|     1|        1|   10|2013|
|2013-10-14|    134.8|   135.59|133.36726|   135.59|12.00575337|    11

In [51]:
sql_df = spark.sql("""
SELECT *, 
      dayofweek(date) AS dayofweek,
      month(date)     AS month,
      year(date)      AS year
FROM btc_sql
""")

In [52]:
sql_df.show(10)

+----------+---------+---------+---------+---------+-----------+------+---------+-----+----+
|      date|     open|     high|      low|    close|     volume|trades|dayofweek|month|year|
+----------+---------+---------+---------+---------+-----------+------+---------+-----+----+
|2013-10-06|    122.0|    122.0|    122.0|    122.0|        0.1|     1|        1|   10|2013|
|2013-10-07|   123.61|   123.61|   123.61|   123.61|        0.1|     1|        2|   10|2013|
|2013-10-08|   123.91|   124.19|    123.9|   124.18|     3.9916|     4|        3|   10|2013|
|2013-10-09|124.01687|124.01687|   123.84|   123.84|      2.823|     3|        4|   10|2013|
|2013-10-10|   125.85|   125.86|   125.85|   125.86|        2.0|     2|        5|   10|2013|
|2013-10-12|    127.5|    127.5|    127.0|    127.0|        4.0|     3|        7|   10|2013|
|2013-10-13| 131.8408| 131.8408| 131.8408| 131.8408|        0.1|     1|        1|   10|2013|
|2013-10-14|    134.8|   135.59|133.36726|   135.59|12.00575337|    11

In [53]:
type(sql_df)

pyspark.sql.dataframe.DataFrame

In [54]:
sql_df.createOrReplaceTempView("btc_sql2")

In [55]:
spark.sql("""
SELECT *
FROM btc_sql2
""").show(10)

+----------+---------+---------+---------+---------+-----------+------+---------+-----+----+
|      date|     open|     high|      low|    close|     volume|trades|dayofweek|month|year|
+----------+---------+---------+---------+---------+-----------+------+---------+-----+----+
|2013-10-06|    122.0|    122.0|    122.0|    122.0|        0.1|     1|        1|   10|2013|
|2013-10-07|   123.61|   123.61|   123.61|   123.61|        0.1|     1|        2|   10|2013|
|2013-10-08|   123.91|   124.19|    123.9|   124.18|     3.9916|     4|        3|   10|2013|
|2013-10-09|124.01687|124.01687|   123.84|   123.84|      2.823|     3|        4|   10|2013|
|2013-10-10|   125.85|   125.86|   125.85|   125.86|        2.0|     2|        5|   10|2013|
|2013-10-12|    127.5|    127.5|    127.0|    127.0|        4.0|     3|        7|   10|2013|
|2013-10-13| 131.8408| 131.8408| 131.8408| 131.8408|        0.1|     1|        1|   10|2013|
|2013-10-14|    134.8|   135.59|133.36726|   135.59|12.00575337|    11

In [56]:
spark.sql("""

SELECT year, 
       ROUND(AVG(open), 2) as avg_open,
       ROUND(AVG(high), 2) as avg_high,
       ROUND(AVG(low),  2) as avg_low,
       ROUND(AVG(close),2) as avg_close,
       ROUND(MIN(close),2) as min_close,
       ROUND(MAX(close),2) as max_close,
       ROUND(AVG(volume),2)as avg_volume,
       ROUND(AVG(trades),2)as avg_trades

FROM btc_sql2
GROUP BY year
ORDER BY AVG(close) DESC

""").show()

+----+--------+--------+--------+---------+---------+---------+----------+----------+
|year|avg_open|avg_high| avg_low|avg_close|min_close|max_close|avg_volume|avg_trades|
+----+--------+--------+--------+---------+---------+---------+----------+----------+
|2021|47378.11|48812.43|45668.61| 47426.09|  29398.9|  67559.8|   5254.15|  47278.85|
|2022| 31267.3|31990.07|30400.86| 31172.53|  18469.2|  47738.3|   3685.18|  27303.61|
|2020|11056.25|11336.65|10783.76| 11116.07|   4850.0|  28959.2|   6386.29|  22526.72|
|2018| 7552.73| 7783.03| 7258.39|  7523.41|   3180.1|  17088.0|   6301.05|  19339.31|
|2019| 7355.56| 7562.66|  7131.1|  7364.96|   3360.3|  12933.7|   6106.85|  17539.89|
|2017| 3936.78| 4118.18| 3767.02|   3972.5|   789.88|  19356.9|   5048.12|  14161.16|
|2016|  566.87|  576.05|  557.45|   568.28|   364.49|    978.0|   1283.67|   1306.76|
|2014|  533.02|  548.64|  515.94|    531.5|   311.01|   919.11|      14.2|    102.54|
|2013|  524.07|  564.18|  484.91|   528.83|    122.0| 

In [58]:
spark.sql("SELECT * FROM btc_sql").show(5)

+----------+---------+---------+------+------+------+------+
|      date|     open|     high|   low| close|volume|trades|
+----------+---------+---------+------+------+------+------+
|2013-10-06|    122.0|    122.0| 122.0| 122.0|   0.1|     1|
|2013-10-07|   123.61|   123.61|123.61|123.61|   0.1|     1|
|2013-10-08|   123.91|   124.19| 123.9|124.18|3.9916|     4|
|2013-10-09|124.01687|124.01687|123.84|123.84| 2.823|     3|
|2013-10-10|   125.85|   125.86|125.85|125.86|   2.0|     2|
+----------+---------+---------+------+------+------+------+
only showing top 5 rows



In [64]:
spark.sql("""
SELECT *, 
    LAG(close) OVER(ORDER BY date) previos_close
    
FROM btc_sql
ORDER BY date
""").show()

22/10/14 00:36:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/10/14 00:36:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/10/14 00:36:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/10/14 00:36:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/10/14 00:36:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+----------+---------+---------+---------+---------+-----------+------+-------------+
|      date|     open|     high|      low|    close|     volume|trades|previos_close|
+--

In [65]:
spark.stop()